In [ ]:
#conda installed
library("TFBSTools")
library("pROC")
library("Matrix")
library("ranger")
library("hash")

#Need to install seperately via Bioconductor 
#BiocManager::install(version = "3.16")
library("glmnet")
library("BSgenome.Hsapiens.UCSC.hg38")
library("BSgenome.Hsapiens.UCSC.hg38.masked")
library("JASPAR2020")
library("foreach")
library("doParallel")

In [ ]:
source("/grid/home/azheng/dsb/PredDSB/script/miscFunctions.R")

In [ ]:
Genome=BSgenome.Hsapiens.UCSC.hg38
model="human"
SeqinfoGenome=seqinfo(BSgenome.Hsapiens.UCSC.hg38)
Chr.V=c(paste0("chr",1:22),"chrX")
SeqInfo=SeqinfoGenome[Chr.V]

# Importing Data

In [ ]:
# OPTION ------------------------------------------------------------
#mode="Epigenome" # To predict using Epigenomic and Chromatin data (ChIP-seq and DNase-seq data)
#mode="EpigenomeForU2OS" # To train a model using Epigenomic and Chromatin data that are available in both NHEK and U2OS cells, i.e.: DNA-seq, CTCF, H3K4me1/3, H3K9me3, H3K27ac, H3K27me3, H3K36me3 and POL2B. 
#mode="Motif" # To predict using DNA motif data only. 
#mode="Motif+Shape" # To predict using DNA motif data and DNA shape. 

mode="Epigenome"

# FILES ------------------------------------------------------------
dire = "/grid/home/azheng/dsb/PredDSB/"
data_dir = paste(dire,"data/",sep="")
output_dir = paste("/grid/home/azheng/dsb/","genome_dsb/",sep="")

# ctype = "lung"
# annot_loc = paste(data_dir,"Epigenome_lung/combined/",sep="")
# fileAnnot=list.files("/grid/home/azheng/dsb/PredDSB/data/Epigenome_lung/combined")
# AnnotNames=as.vector(sapply(fileAnnot,function(x){strsplit(x,'.bed')[[1]][1]}))

ctype = "liver"
annot_loc = "/grid/home/azheng/dsb/PredDSB/data/Epigenome_liver/combined/"
fileAnnot=list.files("/grid/home/azheng/dsb/PredDSB/data/Epigenome_liver/combined")
AnnotNames=as.vector(sapply(fileAnnot,function(x){strsplit(x,'.bed')[[1]][1]}))

#ctype = "NHEK"
# annot_loc = "/grid/home/azheng/dsb/PredDSB/data/Epigenome_hg38/"
# fileAnnot=list.files("/grid/home/azheng/dsb/PredDSB/data/Epigenome_hg38")
# AnnotNames=as.vector(sapply(fileAnnot,function(x){strsplit(x,'_')[[1]][1]}))

# annot_loc = "/grid/home/azheng/dsb/PredDSB/data/h2ax/"
# fileAnnot=list.files("/grid/home/azheng/dsb/PredDSB/data/h2ax")
# AnnotNames=as.vector(sapply(fileAnnot,function(x){strsplit(x,'_')[[1]][1]}))

# Import breaks
fileBedBreaksPos=paste(data_dir,"DSB/breakome_DSBcap_hg38_20kseq.bed",sep="")
fileBedBreaksNeg=paste(data_dir,"DSB/breakome_DSBcap_hg38_20kseq_neg.bed",sep="")

In [ ]:
options(warn=-1)

# Conversion of bed files to GR

In [ ]:
# Import breaks
dataBreaksPos.GR=sort(readGFBed(fileBedBreaksPos,SeqInfo))
dataBreaksNeg.GR=sort(readGFBed(fileBedBreaksNeg,SeqInfo))
dataBreaks.GR=c(dataBreaksPos.GR,dataBreaksNeg.GR)

In [ ]:
flist_bed=list.files("/grid/home/azheng/dsb/PredDSB/data/DSB/chrY.bed")

In [ ]:
hash_fn = hash()
for(i in flist_bed){
    key_fn = "chrY"
    hash_fn[[key_fn]] = i
}
hash_fn

In [ ]:
cluster <- makeCluster(40) 
registerDoParallel(cluster)
d <- foreach(hfn=names(hash_fn), .combine = 'rbind') %dopar% {
    
    source("/grid/home/azheng/dsb/PredDSB/script/miscFunctions.R")
    library("pROC")
    library("Matrix")
    library("ranger")
    library("glmnet")
    library("BSgenome.Hsapiens.UCSC.hg38")
    library("BSgenome.Hsapiens.UCSC.hg38.masked")
    
    fileTestHR=hash_fn[[hfn]]
    fileTestHR.GR=sort(readGFBed(fileTestHR,SeqInfo))
    gr_df = as.data.frame(fileTestHR.GR)
    # Import other data
    GenomicFeatureList.GR=list()
    for(i in 1:length(AnnotNames)){
        GenomicFeatureList.GR[[i]] <- sort(unique(readGFBed(paste0(annot_loc,fileAnnot[i]),SeqInfo)))
        print(paste0(AnnotNames[i]," : ",length(GenomicFeatureList.GR[[i]])))
    }
    names(GenomicFeatureList.GR)=AnnotNames
    #Train
    bin.Mat=c(rep(1,length(dataBreaksPos.GR)),rep(0,length(dataBreaksNeg.GR)))
    for(i in 1:length(GenomicFeatureList.GR)){
        GRi=GenomicFeatureList.GR[[i]]
        annotPosi=annotateLoci(dataBreaksPos.GR,GRi)
        annotNegi=annotateLoci(dataBreaksNeg.GR,GRi)
        annoti=c(annotPosi,annotNegi)
        annoti[annoti>1]=1
        bin.Mat=cbind(bin.Mat,annoti)
        rm(annoti)
        print(paste0(AnnotNames[i]," annotated"))
    }
    colnames(bin.Mat)=c("Breaks",AnnotNames)
    #Test
    test.Mat=c(rep(1,length(fileTestHR.GR)/2),rep(0,length(fileTestHR.GR)/2))
    for(i in 1:length(GenomicFeatureList.GR)){
        GRi=GenomicFeatureList.GR[[i]]
        annotPosi=annotateLoci(fileTestHR.GR,GRi)
        annoti=c(annotPosi)
        annoti[annoti>1]=1
        test.Mat=cbind(test.Mat,annoti)
        rm(annoti)
        print(paste0(AnnotNames[i]," annotated"))
    }
    colnames(test.Mat)=c("Breaks",AnnotNames)
    
    test_mat_df = as.data.frame(test.Mat) 
    gr_df$index <- 1:nrow(gr_df)
    test_mat_df$index <- 1:nrow(test_mat_df)
    new_df <- merge(gr_df, test_mat_df,
                    by = 'index', all = TRUE) 
    rmna_newdf = na.omit(new_df)
    
    dataDSB=data.frame(bin.Mat)
    testDSB=data.frame(test.Mat)
    rownames(dataDSB)=1:nrow(dataDSB)
    rownames(testDSB)=1:nrow(testDSB)
    idxs=sample(1:nrow(dataDSB),3e4)
    dataDSBlearn=dataDSB[sort(idxs),]
    dataDSBtest=na.omit(testDSB)
    
    RFall=ranger("Breaks~.",data=dataDSBlearn,importance="permutation")
    tempdf = predict(RFall,dataDSBtest)$predictions
    tempdf = as.data.frame(tempdf)
    tempdf$index2 <- 1:nrow(tempdf)
    rmna_newdf$index2 <- 1:nrow(rmna_newdf) 
    res_df <- merge(tempdf, rmna_newdf,
                    by = 'index2', all = TRUE) 
    fnam = paste(ctype, hfn, sep = "_")
    fnam = paste(fnam, "csv", sep = ".")
    fnam = paste(output_dir, fnam, sep = "")
    print(fnam)
    write.csv(res_df, fnam)
}

In [ ]:
stopCluster(cluster)

## Prediction

In [ ]:
# PREDICTIONS --------------------------------------------------------------------

#dir.create(paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode))

# Training and testing sets
dataDSB=data.frame(bin.Mat)
rownames(dataDSB)=1:nrow(dataDSB)
idxs=sample(1:nrow(dataDSB),3e4)
dataDSBlearn=dataDSB[sort(idxs),]
dataDSBtest=dataDSB[-idxs,]

# Random Forests
RFall=ranger("Breaks~.",data=dataDSBlearn,importance="permutation")
rocRFall=roc(as.factor(dataDSBtest[,1]),predict(RFall,dataDSBtest)$predictions,ci=T)
aucRF=pROC::auc(rocRFall)

varimp=data.frame(Feature=names(RFall$variable.importance),VariableImportance=RFall$variable.importance)
varimp=varimp[order(varimp[,2],decreasing=T),]
file_varimp=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/varimpRF_",mode,".csv")
write.table(varimp,file=file_varimp,row.names=F,sep='\t',quote=F)

file_rocRF=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/rocRF_",mode,".pdf")
pdf(file_rocRF,4,4)
plot(rocRFall,main=paste0("AUC: ",round(aucRF,4)))
dev.off()

if(mode=="EpigenomeForU2OS"){
 save(RFall,file=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/RF_",mode,"_10vars.RData"))
}

In [ ]:
plot(rocRFall,main=paste0("AUC: ",round(aucRF,4)))

In [ ]:
# Lasso logistic regression
CVLasso=cv.glmnet(as(dataDSBlearn[,-1],"Matrix"),dataDSBlearn[,1],family="binomial",parallel=F)
lambda=CVLasso$lambda.min # CVLasso$lambda.min or CVLasso$lambda.1se
CVLassoError=CVLasso$cvm[which(CVLasso$lambda==lambda)]
devLasso=deviance(CVLasso$glmnet.fit)[which(CVLasso$lambda==lambda)]
coefLasso=CVLasso$glmnet.fit$beta[,which(CVLasso$lambda==lambda)]
coefLassoMat=data.frame(Variable=names(coefLasso),Coefficient=round(coefLasso,5))
write.table(coefLassoMat,file=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/coefLassoMat_",mode,".csv"),row.names=F,sep='\t',quote=F)

rocLasso=roc(as.factor(dataDSBtest[,1]),predict(CVLasso,as(dataDSBtest[,-1],"Matrix")),ci=T)
aucLasso=pROC::auc(rocLasso)

file_rocLasso=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/rocLasso_",mode,".pdf")
pdf(file_rocLasso,4,4)
plot(rocLasso,main=paste0("AUC: ",round(aucLasso,4)))
dev.off()

if(mode=="EpigenomeForU2OS"){
 save(CVLasso,file=paste0("/grid/home/azheng/dsb/PredDSB/results/pred",mode,"/Lasso_",mode,"_10vars.RData"))
}


In [ ]:
plot(rocLasso,main=paste0("AUC: ",round(aucLasso,4)))